## Combinatorial Optimization


***A Tabu Search Algorithm with Short-term Memory for the Knapsack Problem - Solving Multiple Instances***

**Author:** Guilherme Cadori

**Date:** 24/08/2023


#### 1. Reading the instance

In [11]:
# Importing required libraries
import numpy as np

# Reading the target file
# GC: the code template below was provided in class and was adapted by GC (2023-07-09)

filename = "C:\gccadori\Lab5_CO_TabuSearch_STMOnly\Knap_C500I500.dat"

# Creating variables to store data
linha = None
coluna = None
cost = None
rhs = None
matrix = None

# Function to convert values in a line into a list of float values 
def parse_values(line):
    return [float(value) for value in line.strip().split()]

# Reading and extracting data
with open(filename, "r") as file:
    lines = file.readlines()

# Reading the scalar
linha = int(lines[0].strip())
coluna = int(lines[2].strip())

# Reading the vector
cost = np.array(parse_values(lines[5]))

# Reading the matrix
end = 7 + linha
matrix_lines = [line.strip() for line in lines[7:end]]
matrix = np.array([parse_values(line) for line in matrix_lines],dtype=object)

# Reading the RHS values
rhs = np.array(parse_values(lines[end+1]))


#### 2.	Implementing a Tabu Search with Short-Term Memory

In [12]:
# Implementing a Tabu Search with STM
# Added time limit for algorithm running time
def tabuSearchSTM(c, b, A, maxDurationMinutes=5, nonImprovingSolMax=1, tabuSize=1):
    
    # Importing auxiliary package
    import time
    import numpy as np
    
    # Function for creating an initial feasible solution
    def initialSolution(x, c, b, A):
        OF_Value = np.sum(x * c)
        feasibility = np.all(np.sum(x * A, 1) <= b)
        return OF_Value, feasibility
    
    # Function for generating neighbborhood
    def neighborGenerator(x):
        n = len(x)
        neighbors = []
        for i in range(n):
            neighbor = x.copy()
            neighbor[i] = 1 - neighbor[i]
            neighbors.append(neighbor)
        return neighbors
    
    # Creating search algorithm parameters
    x_best = []
    x_current = []
    tabuList = []
    feasibility = False
    n = len(c)
    randomNumber = np.random.default_rng()
    nonImproving = 0
    iterCount = 0
    start_time = time.time()

    # Creating search algorithm main loop
    while feasibility == False and time.time() - start_time < maxDurationMinutes * 60:
        x_current = []
        for i in range(n):
            if randomNumber.random() < 0.5:
                x_current.append(1)
            else:
                x_current.append(0)
        x_best = x_current
        OF_bestValue, feasibility = initialSolution(x_current, c, b, A)
        iterCount += 1

    while nonImproving <= nonImprovingSolMax and time.time() - start_time < maxDurationMinutes * 60:
        neighbors = neighborGenerator(x_current)
        x_bestNeighbor = None
        FO_Value_bestNeighbor = 0
        for neighbor in neighbors:
            if neighbor not in tabuList:
                neighborValue, neighborFeasbility = initialSolution(neighbor, c, b, A)
                if neighborValue > FO_Value_bestNeighbor and neighborFeasbility == True:
                    x_bestNeighbor = neighbor
                    FO_Value_bestNeighbor = neighborValue
        if x_bestNeighbor is None:
            break
        x_current = x_bestNeighbor
        tabuList.append(x_bestNeighbor)
        if len(tabuList) > tabuSize:
            tabuList.pop(0)
        if FO_Value_bestNeighbor > OF_bestValue:
            bestSolution = x_bestNeighbor
            OF_bestValue = FO_Value_bestNeighbor
        else:
            nonImproving = nonImproving + 1
        iterCount += 1

    # Printing the best solution found
    print('Best solution found')
    print('Objective function value:', round(OF_bestValue, 2))
    print('Elapsed time:', round(time.time() - start_time, 2), 'seconds')
    print('Total iterations:', iterCount)

    return


#### 3.	Running the heuristic for the given instance and returning the results

In [13]:
# Importing auxiliary package
import time

# Record starting time
start_time = time.time()

# Running the proposed heuristic for this instance
tabuSearchSTM(c = cost, 
              b = rhs, 
              A = matrix, 
              nonImprovingSolMax = coluna, 
              tabuSize = coluna)

# Record ending time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = round(end_time - start_time, 4)

# Print the elapsed time
print("\nElapsed time:", elapsed_time, "seconds")


Best solution found
Objective function value: 151.37
Elapsed time: 305.87 seconds
Total iterations: 33

Elapsed time: 305.8695 seconds


**Test Run Results**

The best solution found:

- Objective function value: 151.37
- Elapsed time: 305.8695 seconds

### End